# Algorithmic Fairness, Accountability, and Ethics, Spring 2024

## Mandatory Assignment 2: Debiasing data and fairer models

### Group random:
 * Constantin-Bogdan Craciun (cocr@itu.dk)
 * Gino Franco Fazzi (gifa@itu.dk)
 * Veron Hoxha (veho@itu.dk)

#### Imports

In [26]:
from folktables import ACSDataSource, BasicProblem, generate_categories
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import pandas as pd
import scipy
from numba import njit, jit


### Functions

In [39]:
def adult_filter(data):
    """Mimic the filters in place for Adult data.
    Adult documentation notes: Extraction was done by Barry Becker from
    the 1994 Census database. A set of reasonably clean records was extracted
    using the following conditions:
    ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
    """
    df = data
    df = df[df['AGEP'] > 16]
    df = df[df['PINCP'] > 100]
    df = df[df['WKHP'] > 0]
    df = df[df['PWGTP'] >= 1]
    df = df[df["RAC1P"] < 3] ## keep only Whites and African-Americans
    return df

def evaluate_model(estimator, X, y, groups, cv = 5, random_state = 7):
    '''
    Function to evaluate a given estimator using Cross-Validation.
    Arguments: 
     - estimator: sklearn estimator to use. Must contain a fit and predict method.
     - X: data features
     - y: data labels
     - groups: dictionary indicating the group partition
     - cv: how many k folds splits. Default: 5
     - random_state: random seed for the splits
    '''

    # Let's record all fold's results
    cv_results = {}

    # Create an instance of CV splitter
    kf = KFold(n_splits=cv, random_state=random_state, shuffle=True)

    # For each K fold, fit the model and calculate the metrics
    for k, (train_index, test_index) in enumerate(kf.split(X)):

        # Fit model to the specific K split data
        estimator.fit(X[train_index], y[train_index])
        # Predict on the K split test data
        y_hat = estimator.predict(X[test_index])

        # Calculate Scores for each group
        for group, group_idxs in groups.items():

            # Retrieve group ids
            group_id = group_idxs.iloc[test_index]

            # If group is not in results, add it
            if group not in cv_results:
                cv_results[group] = {"f1": [], "accuracy": [], "pos_rate": []}
            
            # Append the k's f1, accuracy and TPR for the given group
            cv_results[group]["f1"].append(f1_score(y[test_index][group_id], y_hat[group_id]))
            cv_results[group]["accuracy"].append(accuracy_score(y[test_index][group_id], y_hat[group_id]))
            cv_results[group]["pos_rate"].append(np.mean(y_hat[group_id]))

    
    # Aggregate CVs
    for group in groups:

        for metric in ["f1", "accuracy", "pos_rate"]:
            avg = np.mean(cv_results[group][metric])
            std = np.std(cv_results[group][metric])
            cv_results[group][f"{metric}_avg"] = avg
            cv_results[group][f"{metric}_std"] = std
            del cv_results[group][metric]

    return pd.DataFrame(cv_results).T


def debias_features(Xs_np, Xs_p, lambda_ = 1):
    """
    Function that applies the proposed geometric method to removes correlations between data and any number of protected variables.
    by He et al. (https://doi.org/10.1145/3375627.3375864)

    """
    @njit(parallel=True)
    def rj(Xs_np, orthbasis):
        return Xs_np - orthbasis @ orthbasis.T @ Xs_np # rj

    assert Xs_np.shape[0]==Xs_p.shape[0]
    
    # Find orthonormal basis of protected features
    orthbasis = scipy.linalg.orth(Xs_p) #px

    # Debias nonprotected features
    #Xs_np_debiased = Xs_np - orthbasis @ orthbasis.T @ Xs_np # rj
    Xs_np_debiased = rj(Xs_np, orthbasis)

    # Return debiased nonprotected features
    return Xs_np_debiased + lambda_ * (Xs_np - Xs_np_debiased)

#### Data preparation

In [3]:
# Seed for reproducibility
seed = 11012008

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)

ACSIncomeNew = BasicProblem(
    features=[
        'AGEP',
        'COW',
        'SCHL',
        'MAR',
        'RELP',
        'WKHP',
        'PWGTP',
        'SEX',
        'RAC1P',
    ],
    target='PINCP',
    target_transform=lambda x: x > 25000,    
    group=['SEX', 'RAC1P'],
    preprocess=adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

definition_df = data_source.get_definitions(download=True)
categories = generate_categories(features=ACSIncomeNew.features, definition_df=definition_df)
features, labels, groups = ACSIncomeNew.df_to_pandas(acs_data, categories=categories, dummies=True)

# Drop the "redundant" columns
features = features.drop(["RAC1P_White alone", 
                          "SEX_Male", 
                          "SCHL_1 or more years of college credit, no degree",  
                          "MAR_Divorced", 
                          "RELP_Adopted son or daughter",
                          'COW_Working without pay in family business or farm' ], axis = 1) 

print("Columns with the protected features:")
for i, f in enumerate(features.columns):
    if ("RAC1P" in f) or ("SEX" in f):
        print("Column ID: %s" %i, "(%s)"%f)
        
features.head()

Columns with the protected features:
Column ID: 54 (SEX_Female)
Column ID: 55 (RAC1P_Black or African American alone)


,AGEP,WKHP,PWGTP,"COW_Employee of a private for-profit company or business, or of an individual, for wages, salary, or commissions","COW_Employee of a private not-for-profit, tax-exempt, or charitable organization",COW_Federal government employee,"COW_Local government employee (city, county, etc.)","COW_Self-employed in own incorporated business, professional practice or farm","COW_Self-employed in own not incorporated business, professional practice, or farm",COW_State government employee,...,RELP_Other nonrelative,RELP_Other relative,RELP_Parent-in-law,RELP_Reference person,RELP_Roomer or boarder,RELP_Son-in-law or daughter-in-law,RELP_Stepson or stepdaughter,RELP_Unmarried partner,SEX_Female,RAC1P_Black or African American alone
0,21,20.0,52,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,65,8.0,33,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,33,40.0,53,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,18,18.0,106,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,27,50.0,23,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Further preparations

We split the data to set aside a hold-out dataset to use only when reporting the final metrics for the models. Furthermore, we scale the features and dibide the dataset according to their features: protected and not protected.

In [4]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features.values, labels.values.reshape(-1), groups, test_size=0.3, random_state=0, shuffle=True)

group_test_dict = {
    'Males': group_test['SEX'] == 1,
    'Females': group_test['SEX'] == 2,
    'Whites': group_test['RAC1P'] == 1,
    'African-Americans': group_test['RAC1P'] == 2
}

group_train_dict = {
    'Males': group_train['SEX'] == 1,
    'Females': group_train['SEX'] == 2,
    'Whites': group_train['RAC1P'] == 1,
    'African-Americans': group_train['RAC1P'] == 2
}

# Scale all features (even OHE)
scaler = StandardScaler().fit(X_train)
Xs_train = scaler.transform(X_train)
Xs_test = scaler.transform(X_test)
n_features = X_train.shape[1]

# Subset for protected and non-protected features
Xs_train_p = Xs_train[:, 54:]
Xs_test_p = Xs_test[:, 54:]
Xs_train_np = Xs_train[:, :54]
Xs_test_np = Xs_test[:, :54]

## Selected Model: Logistic Regression with L2 regularization

In [7]:
model = LogisticRegression(penalty='l2', max_iter=1000)

# 1. Classification model trained on the raw dataset

We begin by training a simple Logistic regression binary classifier on the raw (potentially biased) dataset, and reporting key performance metrics, discrimanated by each protected attribute class: 
- Gender (men vs. women)
- Race (white vs. black)

### Evaluating the model using cross validation.

In [8]:
results = evaluate_model(model, Xs_train_np, y_train, groups = group_train_dict, cv = 5, random_state = seed)

results

,f1_avg,f1_std,accuracy_avg,accuracy_std,pos_rate_avg,pos_rate_std
Males,0.893812,0.001544,0.840187,0.002014,0.768998,0.003264
Females,0.852496,0.003662,0.800923,0.004442,0.722467,0.003388
Whites,0.877512,0.001867,0.823787,0.002244,0.749275,0.002281
African-Americans,0.846723,0.005324,0.794183,0.007066,0.717866,0.008709


# 2. Classification model trained on the "fairer" versions of the dataset

Using He et al.'s geometric method that removes correlations between data and protected variables, we compute a “fairer” (reprojected) version of the dataset. We then use this dataset to train and evaluate a logistic regression classifier.

In [40]:
accuracies = []
f1s = []

for l in np.linspace(0, 1, 11):
    Xs_train_np_debiased = debias_features(Xs_train_np, Xs_train_p, lambda_=l)
    model.fit(Xs_train_np_debiased, y_train)
    y_hat = model.predict(Xs_test_np)
    
    f1s.append(f1_score(y_test, y_hat))
    accuracies.append(accuracy_score(y_test, y_hat))

C:\Users\ginof\AppData\Local\Temp\ipykernel_19444\773352482.py:79: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'C', False, aligned=True), Array(float64, 2, 'A', False, aligned=True))
  return Xs_np - orthbasis @ orthbasis.T @ Xs_np # rj


MemoryError: Allocation failed (probably too large).

In [19]:
pd.DataFrame([np.linspace(0,1,11), accuracies, f1s], index=["Lambda", "Accuracy", "F1"]).T

,Lambda,Accuracy,F1
0,0.0,0.813630,0.869731
1,0.1,0.813939,0.869956
2,0.2,0.814068,0.870011
3,0.3,0.814222,0.870110
4,0.4,0.814402,0.870233
5,0.5,0.814351,0.870164
6,0.6,0.814376,0.870171
7,0.7,0.814608,0.870293
8,0.8,0.814711,0.870342
9,0.9,0.814711,0.870342


### FAIR PCA

In [69]:
class FairPCA:
    def __init__(self, Xs, p_idxs, n_components):
        self.fit(Xs, p_idxs, n_components)

    def fit(self, Xs, p_idxs, n_components):
        # Extract protected features
        Xs_p = Xs[:, p_idxs]

        # Compute projection matrix (U)
        Z = Xs_p
        #Z = Z - Z.mean(0) # Since we alredy standardised everything, there is not much sense in removing the mean
        R = scipy.linalg.null_space(Z.T @ Xs)
        eig_vals, L = scipy.linalg.eig(R.T @ Xs.T @ Xs @ R)
        self.U = R @ L[:, :n_components]

    def project(self, Xs):
        return Xs @ self.U
    
fair_pca = FairPCA(Xs_train, [54, 55], 30)
Xs_train_debiased_PCA = fair_pca.project(Xs_train)
Xs_test_debiased_PCA = fair_pca.project(Xs_test)

In [72]:
model = LogisticRegression(penalty='l2', max_iter=1000)
print('Cross-validation scores:', cross_validate(model, Xs_train_debiased_PCA, y_train, groups=group_train, cv=5, scoring=["f1", "accuracy"]))

Cross-validation scores: {'fit_time': array([0.10342455, 0.10191202, 0.06066394, 0.06963634, 0.07271314]), 'score_time': array([0.00315213, 0.01088214, 0.00299978, 0.00508189, 0.00654531]), 'test_f1': array([0.87064156, 0.86411632, 0.8708615 , 0.86586847, 0.86737354]), 'test_accuracy': array([0.8145 , 0.80375, 0.81375, 0.80675, 0.80925])}
